In [2]:
### Imports and data reading/loading ###

import pandas as pd
import json

# Simplified function for reading JSON files
def read_json(file_name):
    with open(file_name, 'r') as f:
        data = json.load(f)
    return data

# Utilize the simplified function to load JSON data
receipts = read_json('receipts.json')
users = read_json('users.json')
brands = read_json('brands.json')

# Directly convert dictionaries to DataFrames
receipts_df = pd.json_normalize(receipts)
brands_df = pd.json_normalize(brands)
users_df = pd.json_normalize(users)

### Data Preparation ###

# Convert date strings to datetime objects where applicable
date_columns = ['createDate', 'dateScanned', 'modifyDate', 'pointsAwardedDate', 'purchaseDate', 'finishedDate', 'createdDate', 'lastLogin']
for df in [receipts_df, users_df]:
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

# Convert numeric columns to numeric types
numeric_columns = ['bonusPointsEarned', 'pointsEarned', 'purchasedItemCount', 'totalSpent']
receipts_df[numeric_columns] = receipts_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

### Data Checks and Exploration ###

# Function to perform basic checks on DataFrame
def perform_basic_checks(df, id_col, numeric_cols=None, date_cols=None):
    print(f"Total rows: {len(df)}")
    print(f"Unique {id_col} values: {df[id_col].nunique()}")
    print(f"Null {id_col} values: {df[id_col].isna().sum()}")
    if numeric_cols:
        print(df[numeric_cols].describe())
    if date_cols:
        for col in date_cols:
            if col in df.columns:
                print(f"Null {col} values: {df[col].isna().sum()}")

# Perform checks on receipts, brands, and users
perform_basic_checks(receipts_df, '_id', numeric_columns, date_columns)
perform_basic_checks(brands_df, '_id')
perform_basic_checks(users_df, '_id', date_cols=['createdDate', 'lastLogin'])

# Additional specific checks for receipts
print("Number of duplicated receipt records:", receipts_df.duplicated().sum())
print("Number of duplicated user records:", users_df.duplicated().sum())
print("Number of duplicated brand records:", brands_df.duplicated().sum())

# GroupBy operations for aggregated insights
print(receipts_df.groupby('rewardsReceiptStatus').agg({'totalSpent': 'mean', 'purchasedItemCount': 'sum'}))

# Ensure that category code and category have a one-to-one relationship
print(brands_df.groupby('category')['categoryCode'].nunique())

# Check date order logic as needed, example:
print("Number of records with createDate after finishedDate:", sum(receipts_df['createDate'] > receipts_df['finishedDate']))
